In [1]:
from census import Census
from geometry import TargetGeometry, Boundary
from geocode import GeoCode
import yaml
import utils

/Users/wknq25/opt/anaconda3/envs/geocoder_refactor/lib/python3.11/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
with open("../inputs/gen_config.yaml", "r") as f:
    gen_config = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
with open("../inputs/EW_1851_config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

census_config = config["census"]
boundary_config = config["boundaries"]
comparison_config = config["comparison"]

In [4]:
with open("../inputs/gb1900_config.yaml", "r") as f:
    tg_config = yaml.load(f, Loader=yaml.FullLoader)

## Read Census

In [5]:
EW_1851 = Census(year = census_config["year"],
                           country = census_config["country"],
                           fields = census_config["fields"],
                            census_file = census_config["census_file"],
                            read_params = census_config["csv_params"],
                            field_to_clean = census_config["field_to_clean"],
                            subset_field=census_config["subset_field"])

# EW_1851.add_lkup(lkup_file = boundary_config["rsd_lkup"]["lkup_file"], 
#                 fields = boundary_config["rsd_lkup"]["fields"], 
#                 lkup_params= boundary_config["rsd_lkup"]["lkup_params"],
#                 base_uid = boundary_config["rsd_lkup"]["base_uid"]) #remove this

# EW_1851.data[census_config["unique_address_uid"]] = EW_1851.data.groupby([census_config["fields"]["boundary_1"]["val"], 
#                                                                                                     census_config["fields"]["field_to_geocode"]["val"], 
#                                                                                                     boundary_config["boundary_2"]["gis"]["fields"]["uid"]]).ngroup()
EW_1851.data

Reading census
Address {'standardisation_file': '../inputs/icem_street_standardisation.json', 'min_len': 5}
../inputs/icem_street_standardisation.json
5


,RecID,ParID,ConParID,RC,Address
0,1,1,1,1,PRAED STREET
1,2,1,1,1,PRAED STREET
2,3,1,1,1,PRAED STREET
3,4,1,1,1,PRAED STREET
4,5,1,1,1,PRAED STREET
...,...,...,...,...,...
999995,999996,78,27,1,ST JOHN STREET
999996,999997,78,27,1,ST JOHN STREET
999997,999998,78,27,1,ST JOHN STREET
999998,999999,78,27,1,ST JOHN STREET


In [ ]:
# for rc in EW_1851.subsetlist:
#     subset = EW_1851.data[EW_1851.data[EW_1851.subset_field] == rc]
#     subset.to_csv(f"EW_1851/EW_1851_{rc}.tsv", 
#                     **census_config["processed_csv_params"], index=False)
#     subset.drop_duplicates(subset = [census_config["processed_fields_slim"]["uid"]["val"]]).drop(columns=census_config["fields"]["uid"]["val"]).to_csv(f"EW_1851/EW_1851_{rc}_slim.tsv", 
#                                                                                                          **census_config["processed_csv_params"],
#                                                                                                          index=False)

type: Index(['address_uid'], dtype='object')

## Read Boundary data

In [6]:

# this is where i need to create two lists: one of census blocking columns and one of boundary blocking cols;
list_of_boundaries = []
list_of_boundary_fields = []

for boundary, boundary_vals in boundary_config.items():
    print(boundary)

    boundary_obj = Boundary(name = boundary_vals["name"],
                            file_path = boundary_vals["gis"]["file_path"],
                            fields = boundary_vals["gis"]["fields"],
                            read_params = boundary_vals["gis"]["read_params"],
                            write_params = None,
                            processed_fields = None,
                            processed_csv_params = None,
                            )
    boundary_obj.get_geometry_data()


    if boundary_vals["linktype"] == "lkup_to_gis":


        boundary_obj.add_lkup(lkup_file = boundary_vals["lkup"]["lkup_file"], 
                                fields = boundary_vals["lkup"]["fields"], 
                                lkup_params = boundary_vals["lkup"]["lkup_params"],
                                base_link = boundary_vals["gis"]["fields"]["uid"],
                                lkup_link = boundary_vals["lkup"]["fields"]["gis_link"])

        boundary_obj.dissolve_boundaries(boundary_vals["lkup"]["fields"]["census_link"])
        list_of_boundary_fields.append(census_config["fields"][f"{boundary}"]["val"])

    elif boundary_vals["linktype"] == "lkup_to_census":

        boundary_obj.dissolve_boundaries(boundary_vals["gis"]["fields"]["uid"])

        EW_1851.add_lkup(lkup_file = boundary_vals["lkup"]["lkup_file"], 
                                fields = boundary_vals["lkup"]["fields"], 
                                lkup_params = boundary_vals["lkup"]["lkup_params"],
                                base_link = census_config["fields"][f"{boundary}"]["val"],
                                lkup_link = boundary_vals["lkup"]["fields"]["census_link"])

        list_of_boundary_fields.append(boundary_vals["gis"]["fields"]["uid"])
        # census add lookup

    list_of_boundaries.append(boundary_obj)

comb_boundaries = list_of_boundaries[0].merge_boundaries(list_of_boundaries[1:])


list_of_boundary_fields.append(census_config["fields"]["field_to_geocode"]["val"])

boundary_1
boundary_2


In [7]:
EW_1851.data[census_config["unique_address_uid"]] = EW_1851.data.groupby(list_of_boundary_fields).ngroup()

In [8]:
for rc in EW_1851.subsetlist:
    subset = EW_1851.data[EW_1851.data[EW_1851.subset_field] == rc]
    subset.to_csv(f"EW_1851/EW_1851_{rc}.tsv", 
                    **census_config["processed_csv_params"], index=False)
    subset.drop_duplicates(subset = [census_config["processed_fields_slim"]["uid"]["val"]]).drop(columns=census_config["fields"]["uid"]["val"]).to_csv(f"EW_1851/EW_1851_{rc}_slim.tsv", 
                                                                                                         **census_config["processed_csv_params"],
                                                                                                         index=False)

In [ ]:

# PARISHES = Boundary("parishboundaries",
#                fields = boundary_config["parish_gis"]["fields"],

#                 read_params = boundary_config["parish_gis"]["read_params"],
#                 write_params = boundary_config["parish_gis"]["write_params"],
               
#                file_path = boundary_config["parish_gis"]["file_path"],
# )

# PARISHES.get_geometry_data()
# PARISHES.add_lkup(lkup_file = boundary_config["parish_gis_lkup"]["lkup_file"], 
#                   fields = boundary_config["parish_gis_lkup"]["fields"], 
#                   lkup_params = boundary_config["parish_gis_lkup"]["lkup_params"])
# PARISHES.dissolve_boundaries(boundary_config["parish_gis_lkup"]["fields"]["conparid_field"])

# RSD = Boundary("rsdboundaries",
#                fields = boundary_config["rsd_gis"]["fields"],

#                 read_params = boundary_config["rsd_gis"]["read_params"],
#                 write_params = boundary_config["rsd_gis"]["write_params"],
               
#                file_path = boundary_config["rsd_gis"]["file_path"],)

# RSD.get_geometry_data()

# RSD.dissolve_boundaries(boundary_config["rsd_gis"]["fields"]["uid"])

# RSD_PARISHES = PARISHES.merge_boundaries([RSD, ])
# RSD_PARISHES.data

,conparid_51-91,ID,UKDS_ID,CEN_1851,geometry
0,1.0,909,909.0,10001,"POLYGON ((525354.500 181032.703, 525195.030 18..."
1,2.0,925,925.0,10001,"MULTIPOLYGON (((525354.500 181032.703, 525354...."
2,5.0,920,920.0,10001,"MULTIPOLYGON (((524695.062 182656.219, 524679...."
3,11.0,897,897.0,10001,"MULTIPOLYGON (((526474.750 182215.172, 526320...."
4,12.0,878,878.0,10001,"POLYGON ((525713.312 183236.547, 525674.562 18..."
...,...,...,...,...,...
36243,12735.0,8121,8121.0,6230004,"MULTIPOLYGON (((248830.687 390441.094, 248801...."
36244,12736.0,8128,8128.0,6230004,"MULTIPOLYGON (((250147.016 390176.187, 250131...."
36245,12737.0,8116,8116.0,6230004,"MULTIPOLYGON (((246060.312 393420.719, 245992...."
36246,12738.0,8126,8126.0,6230004,"MULTIPOLYGON (((240828.228 390081.534, 240831...."


## Read Target Geometry Dataset

In [9]:
target_geom = TargetGeometry(
                        name = tg_config["name"],
                        file_path = tg_config["file_path"],

                        fields = tg_config["fields"],

                        read_params = tg_config["read_params"],
                        write_params = tg_config["write_params"],
                        field_to_clean = tg_config["field_to_clean"],
                        processed_fields = tg_config["processed_fields_slim"],
                        processed_csv_params = tg_config["processed_csv_params"],
                        


                        )
target_geom.get_geometry_data()
target_geom.assigntoboundary(comb_boundaries.data, )
target_geom.dedup_streets(dedup_fields = [comparison_config["tg_block"], 
                                     tg_config["fields"]["address"]], 
                     uid_field = tg_config["fields"]["uid_field"], 
                     max_points = tg_config["dedup_params"]["max_points_per_unit"], 
                     max_dist_btwn_points = tg_config["dedup_params"]["max_dist_between_points"],)

target_geom.data[tg_config["processed_fields_slim"]["uid"]["val"]] = target_geom.data.groupby(list(utils.flatten([comparison_config["tg_block"],
                                                                                               tg_config["fields"]["uid_field"]])),).ngroup()
target_geom.data


final_text {'standardisation_file': '../inputs/gb1900_standardisation.json', 'min_len': 5}
../inputs/gb1900_standardisation.json
5


/Users/wknq25/Documents/CensusGeocoder/censusgeocoder/utils.py:501: RuntimeWarning: Mean of empty slice.
  mean_dist = spatial.distance.pdist(np.array(list(zip(coords.x, coords.y)))).mean()


108564
107595


,pin_id,final_text,geometry,conparid_51-91,ID,UKDS_ID,CEN_1851,count,dist_calc,pin_id_removed,address_uid
0,5915dfc02c66dc085a00355d,FURNACE WOOD,POINT (570223.823 123089.103),708.0,16827,16827.0,770003,1,NaN,NaN,6205
1,58ee89462c66dcf8fa0d1438,SEVEN ACRE WOOD,POINT (569636.834 121401.775),708.0,16827,16827.0,770003,1,NaN,NaN,6193
2,58de5a672c66dcf8fa089f94,HOGSTY SHAW,POINT (567578.944 116901.086),708.0,16827,16827.0,770003,1,NaN,NaN,6180
3,591487502c66dc085a000fe2,TARGET DISUSED,POINT (569244.857 115110.351),708.0,16827,16827.0,770003,1,NaN,NaN,6204
4,591222672c66dc2d0e002047,HODGKINS FARM,POINT (564275.688 115878.205),708.0,16827,16827.0,770003,1,NaN,NaN,6197
...,...,...,...,...,...,...,...,...,...,...,...
108559,58a76f672c66dc81190b25ca,SCHOOL,POINT (500956.241 354731.349),7459.0,11824,11824.0,4260002,1,NaN,NaN,54011
108560,58b854f72c66dc21b100ca02,BLACK LANE HEAD,POINT (378712.306 425731.224),9095.0,10216,10216.0,4770004,1,NaN,NaN,65743
108561,58aece9a2c66dc81190d8cc4,CHATTISHAM PLACE,POINT (609029.313 242062.757),3273.0,16425,16425.0,2210001,1,NaN,NaN,23495
108562,5843560e2c66dc145f008020,UPTON LODGE,POINT (510372.833 301043.099),2340.0,22462,22462.0,1750002,1,NaN,NaN,18038


## Write Target Geometry Output

In [10]:
target_geom.data.to_csv(f"{EW_1851.country}_{EW_1851.year}/{target_geom.name}_{EW_1851.country}_{EW_1851.year}.tsv", 
                        sep = "\t", 
                        index=False)
target_geom.data.to_csv(f"{EW_1851.country}_{EW_1851.year}/{target_geom.name}_{EW_1851.country}_{EW_1851.year}_slim.tsv",index=False, **target_geom.write_params_slim,
                        )

### Read in Target Geometry processed

In [ ]:
GB1900 = TargetGeometry(EW_1851.country,
                        EW_1851.year,
                        name = "gb1900",
                        file_path = "EW_1851/gb1900_EW_1851_slim.tsv",

                        fields = {"uid_field":"uid", 
                                  "address":"final_text",
                                  "conparid":"conparid_51-91",
                                  "cen":"CEN_1851"},

                        read_params = {"encoding":"utf-8",
                                       "filetype":"csv",
                                       "sep":"\t",
                                       },
                        write_params = {},


                        )

GB1900.data = GB1900.read_geometry_file()
GB1900.data = GB1900.data.set_index("uid").sort_index()
GB1900.data

,final_text,conparid_51-91,CEN_1851
uid,,,
0,PADDINGTON BASIN,1.0,10001
1,RICHMOND ROAD,1.0,10001
2,PADDINGTON,1.0,10002
3,CLIFTON ROAD,1.0,10001
4,CLIFTON GARDENS,1.0,10001
...,...,...,...
892342,CYTTIAUR GWYDDCLOD,12755.0,6230006
892343,CRECRIST BCH,12755.0,6230006
892344,SPEAR HEADS CELT C FOUND A D,12755.0,6230006


In [ ]:
census_config

{'country': 'EW',
 'year': 1851,
 'census_file': '../data/refactor_test_1851.tsv',
 'fields': {'uid': {'RecID': 'int32'},
  'address': {'Address': 'str'},
  'ConsistentParishID': {'ConParID': 'int16'},
  'ParishID': {'ParID': 'int16'},
  'SubsetField': {'RC': 'int8'}},
 'csv_params': {'sep': '\t',
  'encoding': 'latin-1',
  'quoting': 3,
  'na_values': ['.', ' ', '-'],
  'nrows': 1000000},
 'field_to_clean': {'Address': {'standardisation_file': '../inputs/icem_street_standardisation.json',
   'min_len': 5}},
 'subset_field': 'RC',
 'processed_fields_slim': {'address': {'Address': 'str'},
  'ConsistentParishID': {'ConParID': 'int16'},
  'rsd_id': {'CEN_1851': 'int16'}},
 'processed_csv_params': {'sep': '\t', 'encoding': 'utf-8'}}

### Read in processed census and link to processed target geometry

In [ ]:
for rc in EW_1851.subsetlist:

        EW_1851_subset = Census(year = census_config["year"],
                            country = census_config["country"],
                            fields = census_config["processed_fields_slim"],
                                census_file = f"EW_1851/EW_1851_{rc}_slim.tsv",
                                read_params = census_config["processed_csv_params"],)
    
        # EW_1851_subset.data = EW_1851_subset.data.set_index("address_uid")
    
        # geocoded_census = GeoCode(EW_1851_subset, 
        #     GB1900, 
        #     census_block=["CEN_1851", "ConParID"], 
        #       target_geom_block=["CEN_1851", "conparid_51-91"],
        #       comparison_alg = "rapidfuzzy_wratio",
        #       sim_thresh = 0.9)
    
        # geocoded_census.lnked.to_csv(f"{geocoded_census.census.country}_{geocoded_census.census.year}_{geocoded_census.target_geometry.name}_{rc}.tsv",
        #                          sep = "\t",
        #                          index = False
        #                          )
    
        # geocoded_census.lnked_dup.to_csv(f"{geocoded_census.census.country}_{geocoded_census.census.year}_{geocoded_census.target_geometry.name}_{rc}_dup.tsv",
        #                         sep = "\t",
        #                         index = False
        #                         )



Reading census


In [ ]:
EW_1851_subset.data

,ConParID,Address,CEN_1851,address_uid
0,1,PRAED STREET,10001,482
1,1,EDGWARE ROAD,10001,272
2,1,RED LION YARD,10001,497
3,1,RED LION,10001,495
4,1,HARROW ROAD,10001,329
...,...,...,...,...
14383,27,CHARTERHOUSE LANE,8931,14381
14384,27,CHARTERHOUSE COURT,8931,14380
14385,27,CROWN COURT,8931,14385
14386,27,ST JOHN STREET,8931,14387


In [ ]:
geocoded_census.lnked.sort_values("uid")

,address_uid,uid,rapidfuzzy_wratio_s,align,Address,final_text,fs
52,491,0,1.00,16,ST MARYS TERRACE,ST MARYS TERRACE,16.0
36,365,0,0.95,16,N ST MARYS TERRACE,ST MARYS TERRACE,15.2
54,495,1,0.90,3,STABLES CHURCH PLACE,STA,2.7
58,506,1,0.90,3,STANLEY PLACE,STA,2.7
59,508,1,0.90,3,STANROSE PLACE,STA,2.7
...,...,...,...,...,...,...,...
183,1913,97,0.90,3,WELLS PLACE ALBION ROAD,S P,2.7
182,1892,97,0.90,3,THOMAS PLACE,S P,2.7
181,1860,97,0.90,3,ST PETERS PLACE,S P,2.7
178,1820,97,0.90,3,RAVENS PLACE NEW ROAD,S P,2.7


In [ ]:
EW_1851_subset.data

,ParID,ConParID,Address,CEN_1851
address_uid,,,,
433,1,1,PRAED STREET,10001
223,1,1,EDGWARE ROAD,10001
448,1,1,RED LION YARD,10001
446,1,1,RED LION,10001
280,1,1,HARROW ROAD,10001
...,...,...,...,...
1772,6,3,PARADISE PLACE,10006
1875,6,3,SWNILS BUILDINGS,10006
1872,6,3,SWAILS BUILDINGS,10006


In [ ]:
GB1900.data.sort_index()

,final_text,conparid_51-91,CEN_1851
uid,,,
0,ST MARYS TERRACE,1.0,10001
1,STA,1.0,10001
2,SCH,1.0,10001
3,CH,1.0,10001
4,MISN CH,1.0,10001
...,...,...,...
207189,BONC DG,12755.0,6230006
207190,COWYN Y CAPEL SITE OF,12755.0,6230006
207191,P,12755.0,6230006
